# Digital Image processing Exercise 3-4

Amirkabir University of Technology

Dr. Rahmati

by Gholamreza Dar 400131018

Spring 2022

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
sns.set_style('dark')

## Functions

In [ ]:
def rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def bgr(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def disp(img, title='', s=8, vmin=None, vmax=None):
    plt.figure(figsize=(s,s))
    plt.axis('off')
    if vmin is not None and vmax is not None:
        plt.imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.show()

### Fourier Aanalysis

In [ ]:
def fourier_analysis(img):
    fourier_img = cv2.dft(img.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT)
    fourier_img_shift = np.fft.fftshift(fourier_img)
    real = fourier_img_shift[:,:,0]
    imag = fourier_img_shift[:,:,1]
    magnitude = cv2.magnitude(real, imag)
    phase = cv2.phase(real, imag)
    return magnitude, phase


### Apply Fourier Filter

In [ ]:
def apply_fourier_filter(img, mask):
    mask = mask[:, :, np.newaxis]
    img_fourier = cv2.dft(img.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT)
    img_fourier_shift = np.fft.fftshift(img_fourier)
    img_fourier_shift *= mask
    img_fourier_shift_back = np.fft.ifftshift(img_fourier_shift)
    img_fourier_inverse = cv2.idft(img_fourier_shift_back, flags=cv2.DFT_SCALE)

    return img_fourier_inverse

### Plot Magnitude and Phase

In [ ]:
def plot_magnitude_and_phase(img, name, color_space="RGB", vmin=None, vmax=None):

    # Change colorspace
    channel_labels = []
    if color_space == "RGB":
        channel_labels = ["R", "G", "B"]
    elif color_space == "HSV":
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        channel_labels = ["H", "S", "V"]
    elif color_space == "LAB":
        img = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        channel_labels = ["L", "a", "b"]
    elif color_space == "YCBCR":
        img = cv2.cvtColor(img, cv2.COLOR_RGB2YCR_CB)
        channel_labels = ["Y", "Cr", "Cb"]
    
    # Seperate channels
    channel_0 = img[:,:,0]
    channel_1 = img[:,:,1]
    channel_2 = img[:,:,2]
    
    # Fourier analysis on each channel
    mag_0, _ = fourier_analysis(channel_0)
    mag_1, _ = fourier_analysis(channel_1)
    mag_2, _ = fourier_analysis(channel_2)

    # Take log for better visualization
    mag_0 = np.log(mag_0)
    mag_1 = np.log(mag_1)
    mag_2 = np.log(mag_2)

    # Display the magnitude of each channel
    fig, axs = plt.subplots(1,3, figsize=(15,5), constrained_layout=True)
    fig.suptitle(f"'{name}' Fourier Analysis | {color_space}", fontsize=14, y=0.925)

    axs[0].set_axis_off()
    axs[1].set_axis_off()
    axs[2].set_axis_off()

    axs[0].set_title(channel_labels[0])
    axs[1].set_title(channel_labels[1])
    axs[2].set_title(channel_labels[2])

    if vmin is not None and vmax is not None:
        axs[0].imshow(mag_0, cmap='gray', vmin=vmin, vmax=vmax)
        axs[1].imshow(mag_1, cmap='gray', vmin=vmin, vmax=vmax)
        axs[2].imshow(mag_2, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        axs[0].imshow(mag_0, cmap='gray')
        axs[1].imshow(mag_1, cmap='gray')
        axs[2].imshow(mag_2, cmap='gray')
    
    fig.savefig(f'{name}_fourier_analysis_{color_space}.png')
    plt.show()

### Get Mask

In [ ]:
def get_mask(magnitude, threshold=0.1, center_margin_percent=5, blur_mask=31, erode_mask=7, erode_iter=4):
    mask = np.ones(magnitude.shape, dtype=np.float32)
    w, h = magnitude.shape
    m = center_margin_percent / 100.0 / 2.0

    # Preprocess magnitude
    if erode_mask > 0:
        if erode_mask % 2 == 0:
            erode_mask += 1
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
        magnitude = cv2.morphologyEx(magnitude, cv2.MORPH_DILATE, kernel, iterations=erode_iter)

    # Threshold
    threshold/= 10000.0
    mask[magnitude >= threshold*magnitude.max()] = 0
    
    # Always want the center to be 1
    mask[w//2-int(m*w):w//2+int(m*w), h//2-int(m*h):h//2+int(m*h)] = 1

    # Blur the mask to avoid ringing
    if blur_mask > 0:
        if blur_mask % 2 == 0:
            blur_mask += 1
        mask = cv2.GaussianBlur(mask, (blur_mask, blur_mask), 0)

    return mask, magnitude

In [ ]:
# # Cagi_2 mask test
# magnitude, _ = fourier_analysis(cagi_2[:,:,0])
# # disp(np.log(magnitude), 'Magnitude of Channel 0')
# mask, mag = get_mask(magnitude, threshold=45, center_margin_percent=6, blur_mask=20)
# print(mask.max())
# disp(np.log(mag), 'magnitude', vmin=8, vmax=20)
# disp(mask, 'Mask', vmin=0, vmax=1)
# disp(np.log(magnitude*mask+0.0001), 'magnitude masked', vmin=3, vmax=20)

In [ ]:
# # Cagi_1 mask test
# magnitude, _ = fourier_analysis(cagi_1[:,:,0])
# # disp(np.log(magnitude), 'Magnitude of Channel 0')
# mask, mag = get_mask(magnitude, threshold=45, center_margin_percent=6, blur_mask=20)
# print(mask.max())
# disp(mask, 'Mask', vmin=0, vmax=1)
# disp(np.log(mag), 'magnitude', vmin=8, vmax=20)
# disp(np.log(magnitude*mask+0.0001), 'magnitude masked', vmin=3, vmax=20)

In [ ]:
# # Cagi_1 mask test
# # Convert to ycbcr
# cagi_2_ycbcr = cv2.cvtColor(cagi_2, cv2.COLOR_RGB2YCR_CB)
# magnitude, _ = fourier_analysis(cagi_2_ycbcr[:,:,0])
# # disp(np.log(magnitude), 'Magnitude of Channel 0')
# mask, mag = get_mask(magnitude, threshold=10, center_margin_percent=6, blur_mask=20)
# print(mask.max())
# disp(mask, 'Mask', vmin=0, vmax=1)
# disp(np.log(mag), 'magnitude', vmin=8, vmax=20)
# disp(np.log(magnitude*mask+0.0001), 'magnitude masked', vmin=8, vmax=20)

### Display Mask

In [ ]:
def display_mask(img, mask, channel, vmin, vmax):

    magnitude, _ = fourier_analysis(img[:, :, channel])
    mask_norm = mask.astype(np.float32)/255

    fig, axs = plt.subplots(1,3, figsize=(15,5), constrained_layout=True)

    axs[0].set_axis_off()
    axs[1].set_axis_off()
    axs[2].set_axis_off()

    axs[0].set_title('Original')
    axs[1].set_title('Mask')
    axs[2].set_title('Filtered')

    if vmin is not None and vmax is not None:
        axs[0].imshow(np.log(magnitude), cmap='gray', vmin=vmin, vmax=vmax)
        axs[1].imshow(mask_norm, cmap='gray')
        axs[2].imshow(np.log(magnitude*mask_norm+0.0001), cmap='gray', vmin=vmin, vmax=vmax)
    else:
        axs[0].imshow(np.log(magnitude), cmap='gray')
        axs[1].imshow(mask_norm, cmap='gray')
        axs[2].imshow(np.log(magnitude*mask_norm+0.0001), cmap='gray')

### Remove Artifacts

In [ ]:
def remove_artifacts(img, name, color_space="RGB", mask_thresholds=[14, 14, 14], center_margin_percent=[5, 5, 5], blur_mask=[0, 0, 0], erode_mask=[7, 7, 7], erode_iter=[4, 4, 4], vmin=5, vmax=18, figsize_list=[(10,10), (10,10), (15,10)], manual_mask=None):
    
    # Convert img to float32
    img = img.astype(np.float32)

    # Change colorspace
    if color_space == "RGB":
        pass
    elif color_space == "HSV":
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    elif color_space == "LAB":
        img = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    elif color_space == "YCBCR":
        img = cv2.cvtColor(img, cv2.COLOR_RGB2YCR_CB)

    # Make a copy of the image (This will be the filtered image)
    img_result = img.copy()

    magnitudes = []
    magnitude_news = []
    masks = []
    # Make a mask for each channel and filter it based on its mask
    for channel in range(img.shape[2]):
        # shift and fourier Transform on each channel
        img_fourier = cv2.dft(img[:,:,channel], flags=cv2.DFT_COMPLEX_OUTPUT)
        img_fourier_shift = np.fft.fftshift(img_fourier)
        # Get the magnitude and pass it to the mask function
        magnitude = cv2.magnitude(img_fourier_shift[:,:,0], img_fourier_shift[:,:,1])
        magnitudes.append(magnitude)
        # Generate a mask based on the bright spots of magnitude
        mask, magnitude_new = get_mask(magnitude, mask_thresholds[channel], center_margin_percent=center_margin_percent[channel], blur_mask=blur_mask[channel], erode_mask=erode_mask[channel], erode_iter=erode_iter[channel])
        magnitude_news.append(magnitude_new)
        if manual_mask is not None:
            mask = manual_mask
        masks.append(mask)
        # Apply the mask to the fourier image
        img_fourier_shift *= mask[:,:,np.newaxis]
        # ishift and inverse fourier transform on each channel
        img_fourier_shift_back = np.fft.ifftshift(img_fourier_shift)
        img_fourier_inverse = cv2.idft(img_fourier_shift_back, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT)
        # Merge the channels
        img_result[:,:,channel] = img_fourier_inverse
    

    # 1. Plot individual channels
    fig, axs = plt.subplots(3,2, figsize=figsize_list[0], constrained_layout=True)
    for channel in range(img.shape[2]):
        axs[channel, 0].set_axis_off()
        axs[channel, 0].set_title(f'Channel {channel}')
        axs[channel, 0].imshow(img[:,:,channel], cmap='gray')

        axs[channel, 1].set_axis_off()
        axs[channel, 1].set_title(f'Channel {channel} Filtered')
        axs[channel, 1].imshow(img_result[:,:,channel], cmap='gray')
    fig.savefig(f'{name}_Filtering_Channels_{color_space}.png')
    
    # Convert back to RGB
    if color_space == "RGB":
        pass
    elif color_space == "HSV":
        img_result = cv2.cvtColor(img_result, cv2.COLOR_HSV2RGB)
        img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
    elif color_space == "LAB":
        img_result = cv2.cvtColor(img_result, cv2.COLOR_LAB2RGB)
        img = cv2.cvtColor(img, cv2.COLOR_LAB2RGB)
    elif color_space == "YCBCR":
        img_result = cv2.cvtColor(img_result, cv2.COLOR_YCR_CB2RGB)
        img = cv2.cvtColor(img, cv2.COLOR_YCR_CB2RGB)
    
    # Convert back to uint8
    # for channel in range(img.shape[2]):
    #     img[:,:,channel] -= np.min(img[:,:,channel])
    #     img[:,:,channel] /= np.max(img[:,:,channel])
    #     img[:,:,channel] *= 255
    print("img")
    for i in range(3):
        print(np.min(img[:,:,i]), np.max(img[:,:,i]))
    img = img.astype(np.uint8)

    # for channel in range(img_result.shape[2]):
    #     img_result[:,:,channel] -= np.min(img_result[:,:,channel])
    #     img_result[:,:,channel] /= np.max(img_result[:,:,channel])
    #     img_result[:,:,channel] *= 255
    print("img_result")
    for i in range(3):
        print(np.min(img_result[:,:,i]), np.max(img_result[:,:,i]))
    img_result = img_result.clip(0, 255)
    img_result = img_result.astype(np.uint8)
    
    # 2. Plot the full image
    fig, axs = plt.subplots(1,2, figsize=figsize_list[1], constrained_layout=True)
    axs[0].set_axis_off()
    axs[0].set_title(f'"{name}" Original')
    axs[0].imshow(img, cmap='gray')

    axs[1].set_axis_off()
    axs[1].set_title(f'"{name}" Filtered')
    axs[1].imshow(img_result, cmap='gray')
    fig.savefig(f'{name}_Filtered_Image_{color_space}.png')
    
    # 3. Plot the mask
    fig, axs = plt.subplots(3,4, figsize=figsize_list[2], constrained_layout=True)
    for channel in range(img.shape[2]):
        axs[channel, 0].set_axis_off()
        axs[channel, 0].set_title(f'Channel {channel} Fourier')
        axs[channel, 0].imshow(np.log(magnitudes[channel]), cmap='gray', vmin=vmin, vmax=vmax)

        axs[channel, 1].set_axis_off()
        axs[channel, 1].set_title(f'Channel {channel} Fourier Processed')
        axs[channel, 1].imshow(np.log(magnitude_news[channel]), cmap='gray', vmin=vmin, vmax=vmax)

        axs[channel, 2].set_axis_off()
        axs[channel, 2].set_title(f'Channel {channel} Mask')
        axs[channel, 2].imshow(masks[channel], cmap='gray')

        axs[channel, 3].set_axis_off()
        axs[channel, 3].set_title(f'Channel {channel} Fourier Masked')
        axs[channel, 3].imshow(np.log(magnitudes[channel]*masks[channel]+0.00000000001), cmap='gray', vmin=vmin, vmax=vmax)
    fig.savefig(f'{name}_Masking_{color_space}.png')


    return img_result
    

## Load Images

In [ ]:
diagonal_grid_img = rgb(cv2.imread('inputs/P4/a/diagonal_gird.png'))
dot_grid = rgb(cv2.imread('inputs/P4/a/dot_grid.png'))
orthogonal_lines = rgb(cv2.imread('inputs/P4/a/orthogonal_lines.png'))

cagi_1 = rgb(cv2.imread('inputs/P4/b/cagi_1.png'))
cagi_1_mask = cv2.imread('inputs/P4/b/cagi_1_mask.png', 0)
cagi_1_mask_2 = cv2.imread('inputs/P4/b/cagi_1_mask_2.png', 0)
cagi_1_mask_3 = cv2.imread('inputs/P4/b/cagi_1_mask_3.png', 0)
cagi_1_mask_4 = cv2.imread('inputs/P4/b/cagi_1_mask_4.png', 0)
cagi_2_mask = cv2.imread('inputs/P4/b/cagi_2_mask.png', 0)
cagi_2_mask_2 = cv2.imread('inputs/P4/b/cagi_2_mask_2.png', 0)
cagi_2 = rgb(cv2.imread('inputs/P4/b/cagi_2.png'))
cagi_3 = rgb(cv2.imread('inputs/P4/b/cagi_3.png'))
test_rgy = rgb(cv2.imread('inputs/P4/b/test_rgy.png'))
test_rgy_mask = cv2.imread('inputs/P4/b/test_rgy_mask.png', 0)

munker_hearts = rgb(cv2.imread('inputs/P4/c/munker_hearts.png'))
munker_spheres_1 = rgb(cv2.imread('inputs/P4/c/munker_spheres_1.png'))
munker_spheres_2 = rgb(cv2.imread('inputs/P4/c/munker_spheres_2.png'))
munker_zigzag_lines = rgb(cv2.imread('inputs/P4/c/munker_zigzag_lines.png'))

munker_cubes= rgb(cv2.imread('inputs/P4/d/munker_cubes.png'))
munker_hearts_2= rgb(cv2.imread('inputs/P4/d/munker_hears_2.png'))
munker_monspiral= rgb(cv2.imread('inputs/P4/d/munker_monspiral.png'))

## A)

In [ ]:
color_spaces = ["RGB", "HSV", "LAB", "YCBCR"]
vmin = 6
vmax = 19

In [ ]:
for color_space in color_spaces:
    plot_magnitude_and_phase(diagonal_grid_img, 'diagonal_grid', color_space, vmin, vmax)

Best: **Cr** in YCbCr

In [ ]:
for color_space in color_spaces:
    plot_magnitude_and_phase(dot_grid, 'dot_grid', color_space, vmin, vmax)

Best: **Cr** in YCbCr

In [ ]:
for color_space in color_spaces:
    plot_magnitude_and_phase(orthogonal_lines, 'orthogonal_lines', color_space, vmin, vmax)

Best: **a** in Lab

## B)

In [ ]:
color_spaces = ["RGB", "HSV", "LAB", "YCBCR"]
vmin = 5
vmax = 14

In [ ]:
for color_space in color_spaces:
    plot_magnitude_and_phase(cagi_1, 'cagi_1', color_space, vmin, vmax)

Best: **Cr** in YCbCr

In [ ]:
for color_space in color_spaces:
    plot_magnitude_and_phase(cagi_2, 'cagi_2', color_space, vmin, vmax)

Best: **Cr** in YCbCr

In [ ]:
for color_space in color_spaces:
    plot_magnitude_and_phase(cagi_3, 'cagi_3', color_space, vmin, vmax)

Best: **Cr** in YCbCr

### Design a mask

In [ ]:
display_mask(cagi_1, cagi_1_mask, channel=0, vmin=6, vmax=16)

In [ ]:
display_mask(cagi_2, cagi_2_mask_2, channel=0, vmin=6, vmax=16)

### Remove Artifacts

In [ ]:
remove_artifacts(
    test_rgy, "Test Image", "RGB",
    mask_thresholds=[100, 100, 60], 
    center_margin_percent=[3,3,3], 
    blur_mask=[0,0,0],
    erode_mask=[2,2,2],
    erode_iter=[3,3,3],
    vmin=5, vmax=18,
    figsize_list=[(6,10), (10,10), (14,9)],
    manual_mask = None);

#### Cagi 1 RGB

In [ ]:
remove_artifacts(
    cagi_1, "cagi_1", "RGB", 
    mask_thresholds=[45, 45, 45], 
    center_margin_percent=[10, 10, 10], 
    blur_mask=[30, 30, 30],
    erode_mask=[7, 7, 7],
    erode_iter=[4, 4, 4],
    vmin=5, vmax=18,
    figsize_list=[(10,10), (10,10), (17,9)]);

#### Cagi 1 YCbCr

In [ ]:
remove_artifacts(
    cagi_1, "cagi_1", "YCBCR", 
    mask_thresholds=[45, 250, 250], 
    center_margin_percent=[10, 10, 10], 
    blur_mask=[30, 30, 30],
    erode_mask=[7, 7, 7],
    erode_iter=[4, 4, 4],
    vmin=5, vmax=18,
    figsize_list=[(10,10), (10,10), (17,9)]);

#### Cagi 2 YCBCR

In [ ]:
remove_artifacts(
    cagi_2, "cagi_2", "YCBCR", 
    mask_thresholds=[10, 150, 150], 
    center_margin_percent=[20, 8.5, 8.5],
    blur_mask=[70, 20, 20],
    erode_mask=[7, 7, 7],
    erode_iter=[5, 2, 2],
    vmin=8, vmax=18,
    figsize_list=[(10,15), (10,10), (18,15)]);

#### Cagi 3 YCBCR

In [ ]:
remove_artifacts(
    cagi_3, "cagi_3", "YCBCR", 
    mask_thresholds=[50, 150, 200], 
    center_margin_percent=[12, 8, 8],
    blur_mask=[35, 25, 25],
    erode_mask=[9, 7, 7],
    erode_iter=[4, 3, 3],
    vmin=8, vmax=18,
    figsize_list=[(10,11), (10,10), (14,10)]);

#### Munker hearts YCBCR

In [ ]:
remove_artifacts(
    munker_hearts, "Munker Hearts", "YCBCR", 
    mask_thresholds=[12, 90, 90], 
    center_margin_percent=[12, 12, 12],
    blur_mask=[35, 40, 40],
    erode_mask=[9, 7, 7],
    erode_iter=[4, 3, 3],
    vmin=8, vmax=18,
    figsize_list=[(10,11), (10,10), (14,10)]);

#### Munker Spheres 1

In [ ]:
remove_artifacts(
    munker_spheres_1, "Munker Spheres 1", "YCBCR", 
    mask_thresholds=[12, 40, 40], 
    center_margin_percent=[5, 5, 5],
    blur_mask=[10, 15, 25],
    erode_mask=[9, 9, 9],
    erode_iter=[4, 4, 4],
    vmin=8, vmax=18,
    figsize_list=[(10,11), (10,10), (14,10)]);

#### Munker Spheres 2

In [ ]:
remove_artifacts(
    munker_spheres_2, "Munker Spheres 2", "YCBCR", 
    mask_thresholds=[12, 40, 40], 
    center_margin_percent=[5, 5, 5],
    blur_mask=[10, 15, 25],
    erode_mask=[9, 9, 9],
    erode_iter=[4, 4, 4],
    vmin=8, vmax=18,
    figsize_list=[(10,11), (10,10), (14,10)]);

#### Munker zigzag lines

In [ ]:
remove_artifacts(
    munker_zigzag_lines, "Munker Zaigzag lines", "RGB", 
    mask_thresholds=[12, 40, 40], 
    center_margin_percent=[5, 5, 5],
    blur_mask=[20, 20, 20],
    erode_mask=[9, 9, 9],
    erode_iter=[4, 4, 4],
    vmin=8, vmax=18,
    figsize_list=[(10,8), (10,10), (16,7)]);

In [ ]:
remove_artifacts(
    munker_cubes, "Munker Cubes", "YCBCR", 
    mask_thresholds=[12, 40, 40], 
    center_margin_percent=[5, 5, 5],
    blur_mask=[20, 20, 20],
    erode_mask=[9, 9, 9],
    erode_iter=[4, 4, 4],
    vmin=8, vmax=18,
    figsize_list=[(8,8), (10,10), (14,7)]);

In [ ]:
remove_artifacts(
    munker_hearts_2, "Munker Hearts 2", "RGB", 
    mask_thresholds=[12, 40, 40], 
    center_margin_percent=[5, 5, 5],
    blur_mask=[20, 20, 20],
    erode_mask=[9, 9, 9],
    erode_iter=[4, 4, 4],
    vmin=8, vmax=18,
    figsize_list=[(8,13), (10,10), (12,9)]);

In [ ]:
remove_artifacts(
    munker_monspiral, "Munker Monspiral", "RGB", 
    mask_thresholds=[12, 40, 40], 
    center_margin_percent=[5, 5, 5],
    blur_mask=[20, 20, 20],
    erode_mask=[9, 9, 9],
    erode_iter=[4, 4, 4],
    vmin=8, vmax=18,
    figsize_list=[(8,13), (10,10), (12,9)]);

### YCrCb Test

In [ ]:
mask = cv2.imread('inputs/P4/b/cagi_1_mask.png', 0)
disp(mask)

In [ ]:
img = cagi_1.copy()

img_ycbcr = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
img_ycbcr = img.copy()

# Select the best channel
Y_channel = img_ycbcr[:,:,0]
Cr_channel = img_ycbcr[:,:,1]
Cb_channel = img_ycbcr[:,:,2]

filter_channels = [0,1,2]

# Fourier analysis on the best channel
if 0 in filter_channels:
    filtered_channel_0 = apply_fourier_filter(Y_channel, mask)
else:
    filtered_channel_0 = Y_channel.copy()

if 1 in filter_channels:
    filtered_channel_1 = apply_fourier_filter(Cr_channel, mask)
else:
    filtered_channel_1 = Cr_channel.copy()

if 2 in filter_channels:
    filtered_channel_2 = apply_fourier_filter(Cb_channel, mask)
else:
    filtered_channel_2 = Cb_channel.copy()


# Combine the filtered channels 
filtered_img = np.zeros((img.shape[0], img.shape[1], img.shape[2]), dtype=np.float32)

if 0 in filter_channels:
    filtered_img[:,:, 0] = cv2.magnitude(filtered_channel_0[:,:,0], filtered_channel_0[:,:,1])
    filtered_img[:,:, 0] = (filtered_img[:,:, 0]/filtered_img[:,:, 0].max()*255)
else:
    filtered_img[:,:, 0] = filtered_channel_0

if 1 in filter_channels:
    filtered_img[:,:, 1] = cv2.magnitude(filtered_channel_1[:,:,0], filtered_channel_1[:,:,1])
    filtered_img[:,:, 1] = (filtered_img[:,:, 1]/filtered_img[:,:, 1].max()*255)
else:
    filtered_img[:,:, 1] = filtered_channel_1

if 2 in filter_channels:
    filtered_img[:,:, 2] = cv2.magnitude(filtered_channel_2[:,:,0], filtered_channel_2[:,:,1])
    filtered_img[:,:, 2] = (filtered_img[:,:, 2]/filtered_img[:,:, 2].max()*255)
else:
    filtered_img[:,:, 2] = filtered_channel_2

filtered_img = filtered_img.astype(np.uint8)

# filtered_img = cv2.cvtColor(filtered_img, cv2.COLOR_YCrCb2RGB)

# Display the Original and Filtered Images
fig, axs = plt.subplots(1,2, figsize=(15,15), constrained_layout=True)

axs[0].set_axis_off()
axs[1].set_axis_off()

axs[0].set_title('Original ch0')
axs[1].set_title('Filtered ch0')

axs[0].imshow(img_ycbcr[:,:,0], cmap='gray')
# aa = cv2.magnitude(filtered_channel_1[:,:,0], filtered_channel_1[:,:,1])
axs[1].imshow(filtered_img[:, :, 0], cmap='gray')
# cv2.imwrite("bahbahbah.png", filtered_img[:,:,0])

plt.show()

fig, axs = plt.subplots(1,2, figsize=(15,15), constrained_layout=True)

axs[0].set_axis_off()
axs[1].set_axis_off()

axs[0].set_title('Original ch1')
axs[1].set_title('Filtered ch1')

axs[0].imshow(img_ycbcr[:,:,1], cmap='gray')
# aa = cv2.magnitude(filtered_channel_1[:,:,0], filtered_channel_1[:,:,1])
axs[1].imshow(filtered_img[:, :, 1], cmap='gray')
# cv2.imwrite("bahbahbah.png", filtered_img[:,:,0])

plt.show()

fig, axs = plt.subplots(1,2, figsize=(15,15), constrained_layout=True)

axs[0].set_axis_off()
axs[1].set_axis_off()

axs[0].set_title('Original ch2')
axs[1].set_title('Filtered ch2')

axs[0].imshow(img_ycbcr[:,:,2], cmap='gray')
# aa = cv2.magnitude(filtered_channel_1[:,:,0], filtered_channel_1[:,:,1])
axs[1].imshow(filtered_img[:, :, 2], cmap='gray')
# cv2.imwrite("bahbahbah.png", filtered_img[:,:,0])

plt.show()

fig, axs = plt.subplots(1,2, figsize=(15,15), constrained_layout=True)

axs[0].set_axis_off()
axs[1].set_axis_off()

axs[0].set_title('Original Image')
axs[1].set_title('Filtered Image')

axs[0].imshow(img_ycbcr[:,:,:], cmap='gray')
# aa = cv2.magnitude(filtered_channel_1[:,:,0], filtered_channel_1[:,:,1])
axs[1].imshow(filtered_img[:, :, :], cmap='gray')
# cv2.imwrite("bahbahbah.png", filtered_img[:,:,0])

plt.show()
    

In [ ]:
disp(filtered_img)

In [ ]:
# Load image
cagi = cagi_1.copy().astype(np.float32)

# Convert to YCrCb
cagi_ycbcr = cv2.cvtColor(cagi, cv2.COLOR_RGB2YCrCb)
cagi_ycbcr_org = cagi_ycbcr.copy()

# Contrast stretching on cagi
for i in range(3):
    cagi[:,:,i] -= cagi[:,:,i].min()
    cagi[:,:,i] /= cagi[:,:,i].max()
cagi *= 255
cagi = cagi.astype(np.uint8)

# Convert back to RGB
cagi_ycbcr = cv2.cvtColor(cagi_ycbcr, cv2.COLOR_YCrCb2RGB)

# Contrast stretching on cagi_ycbcr
for i in range(3):
    cagi_ycbcr[:,:,i] -= cagi_ycbcr[:,:,i].min()
    cagi_ycbcr[:,:,i] /= cagi_ycbcr[:,:,i].max()
cagi_ycbcr *= 255
cagi_ycbcr = cagi_ycbcr.astype(np.uint8)


disp(cagi)
disp(cagi_ycbcr)

In [ ]:
for i in range(3):
    print(cagi[:,:,i].min(), cagi[:,:,i].max())

In [ ]:
for i in range(3):
    print(cagi_ycbcr_org[:,:,i].min(), cagi_ycbcr_org[:,:,i].max())